# SPX Weekly Extension
## Description
This script will be used to simply find the weeks where the S&P500 had the largest extension from it's weekly 9EMA.

In [2]:
import datetime as dt
import numpy as np
import pandas as pd

In [3]:
#Read in weekly bars from csv
df = pd.read_csv("SPX_Weekly.txt")

In [4]:
#Convert strings of date and time to datetime format and set as index
df["datetime"] = df["Date"] + " " + df["Time"]
df["datetime"] =pd.to_datetime(df["datetime"])
#Add ticker column for grouping
df['ticker'] = 'SPX'
df = df.set_index('datetime')

In [5]:
#Create custom indicators
ema9 = lambda x: x.ewm(span=9, min_periods=9, adjust=False, ignore_na=False).mean()

In [6]:
#Apply custom indicators and creat ema9 column
df = df.assign(ema9=df.groupby('ticker')['Close'].apply(ema9))
df = df.dropna(subset=['ema9'])

In [7]:
#Create differences from ema9 to determine extension
df['ema9_diff'] = df['Close'] - df['ema9']
df['ema9_diff%'] = round(df['ema9_diff'] / df['Close'] * 100, 2)

In [8]:
#drop unneccessary columns
df = df.drop(columns=['Date', 'Time', 'Vol', 'OI', 'Vol.1','VolAvg', 'ticker'])

In [9]:
downwardExt = df.sort_values('ema9_diff%', ascending=True)
upwardExt = df.sort_values('ema9_diff%', ascending=False)

In [10]:
upwardExt.head(50)
upwardExt.to_csv('upwardExt.csv')

In [11]:
downwardExt.head(50)
downwardExt.to_csv('downwardExt.csv')